Learn to Determine, Classify the Score (pass/fail) for a Jannah.io Boot Deployment Using Log Files - TextVectorization

In [5]:
#papermill_description=Import_Python_Libraries
import pathlib
import random
import datetime
import time
import collections

import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

In [6]:
#papermill_description=Determine_Directory_Paths_For_Sorting_Log_Files
Jannah_Config = provisioner["inventory"]["group_vars"]["all"]["Jannah"]
Work_Dir = provisioner["inventory"]["group_vars"]["all"]["Jannah"]['global']['ansible']['working_dir']
MOLECULE_EPHEMERAL_DIRECTORY = provisioner["env"]["MOLECULE_EPHEMERAL_DIRECTORY"]
Logs_input = f"{Work_Dir}/logs/TextVectorization"
Logs_train_input = f"{Logs_input}/train"
Logs_test_input = f"{Logs_input}/test"
Logs_train_input_pass_deployment = f"{Logs_train_input}/pass"
Logs_train_input_fail_deployment = f"{Logs_train_input}/fail"
Logs_test_input_pass_deployment = f"{Logs_test_input}/pass"
Logs_test_input_fail_deployment = f"{Logs_test_input}/fail"
sorted_deployment_log_paths = {
    "train": {
                "pass": Logs_train_input_pass_deployment,
                "fail": Logs_train_input_fail_deployment
    },
    "test": {
            "pass": Logs_test_input_pass_deployment,
            "fail": Logs_test_input_fail_deployment
    }
}
deploy_to_docker_desktop_before_training = True
deploy_to_kind_cluster_before_training = True
_ANSIBLE_VERBOSE_LEVEL="-vvvvv"
_MODEL_EPOCHS = 3
_BATCH_SIZE = 3

NameError: name 'provisioner' is not defined

Run Kind Cluster Matrix

In [ ]:
#papermill_description=Generate_Log_File_Name
current_datetime = datetime.datetime.fromtimestamp(time.time()).isoformat().replace(":","-").replace(".","-")
!mkdir -vp $Logs_input
deploy_mode = "deploy-to-kind-cluster-full-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
print(log_file)

In [ ]:
#papermill_description=make_deploy_to_kind_cluster_full_ubuntu_dev_mode
if deploy_to_kind_cluster_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_kind_cluster_local_ubuntu_dev_mode
deploy_mode = "deploy-to-kind-cluster-local-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
if deploy_to_kind_cluster_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_kind_cluster_standalone_ubuntu_dev_mode
deploy_mode = "deploy-to-kind-cluster-standalone-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
if deploy_to_kind_cluster_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=jannah_deployment_with_ansible
deploy_mode = "jannah-deployment-with-ansible"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
if deploy_to_kind_cluster_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

Run Docker Desktop Matrix

In [7]:
#papermill_description=kubectl_config_set_context_docker_desktop
!kubectl config set-context docker-desktop

Context "docker-desktop" modified.


In [ ]:
#papermill_description=deploy_to_docker_desktop_full_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-full-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
if deploy_to_docker_desktop_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_docker_desktop_local_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-local-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
if deploy_to_docker_desktop_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=deploy_to_docker_desktop_standalone_ubuntu_dev_mode
deploy_mode = "deploy-to-docker-desktop-standalone-ubuntu-dev-mode"
log_file = f"{Logs_input}/{deploy_mode}-{current_datetime}.txt"
if deploy_to_docker_desktop_before_training:
    !touch $log_file
    print(log_file)
    !pushd $Work_Dir && ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make $deploy_mode >> $log_file && popd

In [ ]:
#papermill_description=Create_Directories_For_Sorting_Log_Files
!mkdir -vp $Logs_train_input_pass_deployment/
!mkdir -vp $Logs_train_input_fail_deployment/
!mkdir -vp $Logs_test_input_pass_deployment/
!mkdir -vp $Logs_test_input_fail_deployment

In [ ]:
#papermill_description=Copy_Log_Files_For_Processing
!cp -rp ~/jannah-operator/*.log $Logs_input/

In [ ]:
#papermill_description=Glob_Log_Files
log_files  = pathlib.Path(Logs_input).glob("*.log")

In [ ]:
#TODO: Note: To increase the difficulty of the classification problem, replace occurrences of the phrases "All assertions passed", "Assertion failed" from buffer
#      and write buffer back to file.

In [ ]:
#papermill_description=Determine_Pass_or_Fail_Status_For_Each_Deployment_File
_FILE_BUF = ""
pass_deployments = []
failed_deployments = []
for _file in sorted((log_files)):
    with _file.open() as fd:
        _FILE_BUF = fd.read()
        if "All assertions passed" in _FILE_BUF and "Assertion failed" not in _FILE_BUF:
            pass_deployments.append(_file)
        else:
            failed_deployments.append(_file)

In [ ]:
#papermill_description=Print_Pass_Deployment_Counts
len(pass_deployments)

In [ ]:
#papermill_description=Print_Failed_Deployment_Counts
len(failed_deployments)

In [ ]:
#papermill_description=Sort_Pass_Deployments_Into_Train_or_Test_Directories
import random
for _file in pass_deployments:
    train_or_test_group = random.choices(population=['train','test'], weights=[80,20]).pop()
    _new_file_name = _file.name.replace(".log",".txt")
    _new_file = f"{sorted_deployment_log_paths[train_or_test_group]['pass']}/{_new_file_name}"
    _file.rename(_new_file)

In [ ]:
#papermill_description=Sort_Fail_Deployments_Into_Train_or_Test_Directories
import random
for _file in failed_deployments:
    train_or_test_group = random.choices(population=['train','test'], weights=[80,20]).pop()
    _new_file_name = _file.name.replace(".log",".txt")
    _new_file = f"{sorted_deployment_log_paths[train_or_test_group]['fail']}/{_new_file_name}"
    _file.rename(_new_file)

In [ ]:
#papermill_description=Text_Dataset_From_Directory
train_dir=Logs_train_input
batch_size = _BATCH_SIZE
#batch_size = 32
seed = 80

raw_train_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

In [ ]:
#papermill_description=Print_a_Sample_Line
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(_BATCH_SIZE):
    print("Log Line: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

In [ ]:
#papermill_description=Print_the_Labels
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

In [ ]:
#papermill_description=Create_a_validation_set.
raw_val_ds = utils.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

In [ ]:
#papermill_description=Create_a_test_set
test_dir = Logs_test_input
raw_test_ds = utils.text_dataset_from_directory(
    test_dir,
    batch_size=batch_size)

In [ ]:
#papermill_description=Configure_the_datasets_for_performance
raw_train_ds = raw_train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
raw_val_ds = raw_val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
raw_test_ds = raw_test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)


First, use the `'binary'` vectorization mode to build a bag-of-words model. Then use the `'int'` mode with a 1D ConvNet.

In [ ]:
#papermill_description=Vectorization_binary_mode_to_build_a_bag_of_words_model
VOCAB_SIZE = 10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

For the `'int'` mode, in addition to maximum vocabulary size, you need to set an explicit maximum sequence length (`MAX_SEQUENCE_LENGTH`), which will cause the layer to pad or truncate sequences to exactly `output_sequence_length` values:

In [ ]:
#papermill_description=Vectorization_int_mode_to_build_a_bag_of_words_model
MAX_SEQUENCE_LENGTH = 250

int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

Next, call `TextVectorization.adapt` to fit the state of the preprocessing layer to the dataset. This will cause the model to build an index of strings to integers.

Note: It's important to only use your training data when calling `TextVectorization.adapt`, as using the test set would leak information.

In [ ]:
#papermill_description=ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make_a_text_only_dataset_without_labels_then_call_TextVectorization_adapt
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

Print the result of using these layers to preprocess data:

In [ ]:
#papermill_description=Retrieve_a_batch_log_files_and_labels_from_the_dataset.
text_batch, label_batch = next(iter(raw_train_ds))
first_log_file, first_label = text_batch[0], label_batch[0]
print("Log File", first_log_file)
print("Label:", first_label)

The binary vectorization layer returns a multi-hot vector, with a 1 in the location for each token that was in the input string.

In [ ]:
#papermill_description=Print_binary_vectorized_log_lines.
print("'binary' vectorized Log Files:",
      list(binary_vectorize_layer(first_log_file).numpy()))

plt.plot(binary_vectorize_layer(first_log_file).numpy())
plt.xlim(0,1000)

In [ ]:
#papermill_description=Print_int_vectorized_log_lines.
print("'int' vectorized Log Files:",
      int_vectorize_layer(first_log_file).numpy())

In [ ]:
#papermill_description=Print_binary_vectorize_layer_int_vectorize_layer
print("binary_vectorize_layer[0:99] ---> ", binary_vectorize_layer.get_vocabulary()[:99])
print("int_vectorize_layer[0:99] ---> ", int_vectorize_layer.get_vocabulary()[:99])
print("binary_vectorize_layer size: {}".format(len(binary_vectorize_layer.get_vocabulary())))
print("int_vectorize_layer size: {}".format(len(int_vectorize_layer.get_vocabulary())))

### Train the model

It's time to create your neural network.

For the `'binary'` vectorized data, define a simple bag-of-words linear model, then configure and train it:

In [ ]:
#papermill_description=Train_the_binary_model
binary_model = tf.keras.Sequential([
    binary_vectorize_layer,
    layers.Dense(4)])

binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

tf.keras.utils.plot_model(binary_model, show_shapes=True)

In [ ]:
#papermill_description=load_previously_saved_model_if_any
import os
if os.path.isdir("JannahBootTextVectorizationSavedModel.tf"): 
    print("read previously saved model")
    loaded = tf.saved_model.load('JannahBootTextVectorizationSavedModel.tf')

In [ ]:
#papermill_description=Train_the_binary_model_bin_history_fit
bin_history = binary_model.fit(
    raw_train_ds, validation_data=raw_val_ds, epochs=_MODEL_EPOCHS)

print()

Next, you will use the `'int'` vectorized layer to build a 1D ConvNet:


In [ ]:
#papermill_description=def_create_model
def create_model(vocab_size, num_labels, vectorizer=None):
  my_layers =[]
  if vectorizer is not None:
    my_layers = [vectorizer]

  my_layers.extend([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Dropout(0.5),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])

  model = tf.keras.Sequential(my_layers)
  return model

In [ ]:
#papermill_description=int_model_create
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4, vectorizer=int_vectorize_layer)

tf.keras.utils.plot_model(int_model, show_shapes=True)

In [ ]:
#papermill_description=int_model_compile
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
int_history = int_model.fit(raw_train_ds, validation_data=raw_val_ds, epochs=_MODEL_EPOCHS)

In [ ]:
#papermill_description=int_model_compile
loss = plt.plot(bin_history.epoch, bin_history.history['loss'], label='bin-loss')
plt.plot(bin_history.epoch, bin_history.history['val_loss'], '--', color=loss[0].get_color(), label='bin-val_loss')

loss = plt.plot(int_history.epoch, int_history.history['loss'], label='int-loss')
plt.plot(int_history.epoch, int_history.history['val_loss'], '--', color=loss[0].get_color(), label='int-val_loss')

plt.legend()
plt.xlabel('Epoch')
plt.ylabel('CE/token')

You are nearly ready to train your model.

As a final preprocessing step, you will apply the `TextVectorization` layers you created earlier to the training, validation, and test sets:

In [ ]:
#papermill_description=apply_text_vectorization
binary_train_ds = raw_train_ds.map(lambda x,y: (binary_vectorize_layer(x), y))
binary_val_ds = raw_val_ds.map(lambda x,y: (binary_vectorize_layer(x), y))
binary_test_ds = raw_test_ds.map(lambda x,y: (binary_vectorize_layer(x), y))

int_train_ds = raw_train_ds.map(lambda x,y: (int_vectorize_layer(x), y))
int_val_ds = raw_val_ds.map(lambda x,y: (int_vectorize_layer(x), y))
int_test_ds = raw_test_ds.map(lambda x,y: (int_vectorize_layer(x), y))

### Export the model

In [ ]:
#papermill_description=export_bin_tf
binary_model.export('JannahBootTextVectorizationSavedModel.tf')

In [ ]:
#papermill_description=saved_model
loaded = tf.saved_model.load('JannahBootTextVectorizationSavedModel.tf')

In [ ]:
#papermill_description=binary_model_predict_sample_Line
for text_batch, label_batch in raw_test_ds.take(1):
  for i in range(_BATCH_SIZE):
    print("Log Line: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])
    binary_model.predict([text_batch.numpy()[i]])

In [ ]:
#papermill_description=binary_model_serve_sample_Line
for text_batch, label_batch in raw_test_ds.take(1):
  for i in range(_BATCH_SIZE):
    print("Log Line: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])
    loaded.serve(tf.constant([text_batch.numpy()[i]])).numpy()

Learn to Determine, Classify the Score (pass/fail) for a Jannah.io Boot Deployment Using Log Files - TextLineDataset

In [ ]:
#papermill_description=train_and_test_files
TRAIN_FILE_NAMES = []
TEST_FILE_NAMES = []
TRAIN_FILE_NAMES.extend(pathlib.Path(Logs_input+"/train/pass").glob("*.txt"))
TRAIN_FILE_NAMES.extend(pathlib.Path(Logs_input+"/train/fail").glob("*.txt"))

### Load the dataset

Previously, with `tf.keras.utils.text_dataset_from_directory` all contents of a file were treated as a single example. Here, you will use `tf.data.TextLineDataset`, which is designed to create a `tf.data.Dataset` from a text file where each example is a line of text from the original file. `TextLineDataset` is useful for text data that is primarily line-based (for example, poetry or error logs).

Iterate through these files, loading each one into its own dataset. Each example needs to be individually labeled, so use `Dataset.map` to apply a labeler function to each one. This will iterate over every example in the dataset, returning (`example, label`) pairs.

In [ ]:
#papermill_description=define_labeler
def labeler(example, index):
  return example, tf.cast(index, tf.int64)

In [ ]:
#papermill_description=read_text_line_dataset
labeled_data_sets = []

for i, file_name in enumerate(TRAIN_FILE_NAMES):
  lines_dataset = tf.data.TextLineDataset(file_name)
  labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
  labeled_data_sets.append(labeled_dataset)

Next, you'll combine these labeled datasets into a single dataset using `Dataset.concatenate`, and shuffle it with `Dataset.shuffle`:


In [ ]:
#papermill_description=set_buffer_size
BUFFER_SIZE = 50000
BATCH_SIZE = 64
VALIDATION_SIZE = 5000

In [ ]:
#papermill_description=all_labeled_data_concatenate
all_labeled_data = labeled_data_sets[0]
for labeled_dataset in labeled_data_sets[1:]:
  all_labeled_data = all_labeled_data.concatenate(labeled_dataset)

all_labeled_data = all_labeled_data.shuffle(
    BUFFER_SIZE, reshuffle_each_iteration=False)

Print out a few examples as before. The dataset hasn't been batched yet, hence each entry in `all_labeled_data` corresponds to one data point:


In [ ]:
for text, label in all_labeled_data.take(10):
  print("Log Line: ", text.numpy())
  print("Label:", label.numpy())

### Prepare the dataset for training

Instead of using `tf.keras.layers.TextVectorization` to preprocess the text dataset, you will now use the lower-level TensorFlow Text APIs to standardize and tokenize the data, build a vocabulary and use `tf.lookup.StaticVocabularyTable` to map tokens to integers to feed to the model. (Learn more about [TensorFlow Text](https://www.tensorflow.org/text)).

Define a function to convert the text to lower-case and tokenize it:

- TensorFlow Text provides various tokenizers. In this example, you will use the `text.UnicodeScriptTokenizer` to tokenize the dataset.
- You will use `Dataset.map` to apply the tokenization to the dataset.

In [ ]:
#papermill_description=define_jannahbootlogtokenizer
class JannahBootLogTokenizer(tf.keras.layers.Layer):
  def __init__(self):
    super().__init__()
    self.tokenizer = tf_text.UnicodeScriptTokenizer()

  def call(self, text):
    lower_case = tf_text.case_fold_utf8(text)
    result = self.tokenizer.tokenize(lower_case)
    # If you pass a batch of strings, it will return a RaggedTensor.
    if isinstance(result, tf.RaggedTensor):
      # Convert to dense 0-padded.
      result = result.to_tensor()
    return result

In [ ]:
#papermill_description=instantiate_jannahbootlogtokenizer
tokenizer = JannahBootLogTokenizer()

In [ ]:
#papermill_description=tokenized_ds
tokenized_ds = all_labeled_data.map(lambda text, label: (tokenizer(text), label))
tokenized_ds

You can iterate over the dataset and print out a few tokenized examples:

In [ ]:
#papermill_description=print_tokens
for tokens, label in tokenized_ds.take(1):
  break

print(tokens)
print()
print(label)

Next, you will build a vocabulary by sorting tokens by frequency and keeping the top `VOCAB_SIZE` tokens:

In [ ]:
#papermill_description=build_vocabulary 
import collections
tokenized_ds = tokenized_ds.cache().prefetch(tf.data.AUTOTUNE)

vocab_count = collections.Counter()
for toks, labels in tokenized_ds.ragged_batch(1000):
  toks = tf.reshape(toks, [-1])
  for tok in toks.numpy():
    vocab_count[tok] += 1

vocab = [tok for tok, count in vocab_count.most_common(VOCAB_SIZE)]

print("First five vocab entries:", vocab[:5])
print()

To convert the tokens into integers, use the `vocab` set to create a `tf.lookup.StaticVocabularyTable`. You will map tokens to integers with `0` reserved for padding, and `n+1` reserved to denote an out-of-vocabulary (OOV) token.

In [ ]:
#papermill_description=define_vocabulary_table
class JannahBootLogsVocabTable(tf.keras.layers.Layer):
  def __init__(self, vocab):
    super().__init__()
    self.keys = [''] + vocab
    self.values = range(len(self.keys))

    self.init = tf.lookup.KeyValueTensorInitializer(
        self.keys, self.values, key_dtype=tf.string, value_dtype=tf.int64)

    num_oov_buckets = 1

    self.table = tf.lookup.StaticVocabularyTable(self.init, num_oov_buckets)

  def call(self, x):
    result = self.table.lookup(x)
    return result

Try it on a dummy vocabulary:

In [ ]:
#papermill_description=instantiate_vocabulary_table_with_dummy_data
vocab_table = JannahBootLogsVocabTable(['a','b','c'])
vocab_table(tf.constant([''] + list('abcdefghi')))

Create one for the real vocabulary:

In [ ]:
#papermill_description=instantiate_vocabulary_table_with_log_data
vocab_table = JannahBootLogsVocabTable(vocab)

Finally, define a layer to standardize, tokenize and vectorize the dataset using the tokenizer and lookup table:

In [ ]:
#papermill_description=define_layer_to_standardize_tokenize_and_vectorize_the_dataset
preprocess_text = tf.keras.Sequential([
    tokenizer,
    vocab_table
])

You can try this on a single example to print the output:

In [ ]:
#papermill_description=define_layer_to_standardize_tokenize_and_vectorize_the_dataset
example_text, example_label = next(iter(all_labeled_data))
print("Log Line: ", example_text.numpy())
vectorized_text = preprocess_text(example_text)
print("Vectorized log line: ", vectorized_text.numpy())

Now create a dataset pipeline that will process the text on the fly using `Dataset.map`:

In [ ]:
#papermill_description=print_all_encoded_data
all_encoded_data = all_labeled_data.map(lambda text, labels:(preprocess_text(text), labels))

for ids, label in all_encoded_data.take(1):
  break

print("Ids: ", ids.numpy())
print("Label: ", label.numpy())

### Split the dataset into training and test sets

The Keras `TextVectorization` layer also batches and pads the vectorized data. Padding is required because the examples inside of a batch need to be the same size and shape, but the examples in these datasets are not all the same size—each line of text has a different number of words.

`tf.data.Dataset` supports splitting and padded-batching datasets:

In [ ]:
#papermill_description=split_the_dataset_into_training_and_test_sets
train_data = all_encoded_data.skip(VALIDATION_SIZE).shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE)
validation_data = all_encoded_data.take(VALIDATION_SIZE).padded_batch(BATCH_SIZE)

Now, `validation_data` and `train_data` are not collections of (`example, label`) pairs, but collections of batches. Each batch is a pair of (*many examples*, *many labels*) represented as arrays.

To illustrate this:

In [ ]:
#papermill_description=print_text_batch_label_batch
sample_text, sample_labels = next(iter(validation_data))
print("Text batch shape: ", sample_text.shape)
print("Label batch shape: ", sample_labels.shape)
print("First text example: ", sample_text[0])
print("First label example: ", sample_labels[0])

Configure the datasets for better performance as before:

In [ ]:

#papermill_description=autotune_train_data_validation_data
train_data = train_data.prefetch(tf.data.AUTOTUNE)
validation_data = validation_data.prefetch(tf.data.AUTOTUNE)

### Train the model

You can train a model on this dataset as before:

Since this text vectorization adds `0` for padding and `n+1` for out-of-vocabulary (OOV) tokens, the vocabulary size has increased by two:

In [ ]:
#papermill_description=create_and_compile_model
model = create_model(vocab_size=VOCAB_SIZE+2, num_labels=300)

model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
#papermill_description=plot_model
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
#papermill_description=fit_model
history = model.fit(train_data, validation_data=validation_data, epochs=_MODEL_EPOCHS)

In [ ]:
#papermill_description=evaluate_model
metrics = model.evaluate(validation_data, return_dict=True)

print("Loss: ", metrics['loss'])
print("Accuracy: {:2.2%}".format(metrics['accuracy']))

### Export the model

To ANSIBLE_VERBOSE_LEVEL=$_ANSIBLE_VERBOSE_LEVEL make the model capable of taking raw strings as input, pack both the text processor and the model into a `keras.Sequential`:

In [ ]:
#papermill_description=sequential_model
export_model = tf.keras.Sequential([
    preprocess_text,
    model
])

In [ ]:
#papermill_description=sequential_model_compile
export_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

This model you can run directly on batches of strings:

In [ ]:
#papermill_description=create_a_test_dataset_of_raw_strings.
test_ds = all_labeled_data.take(VALIDATION_SIZE).batch(BATCH_SIZE)
test_ds = test_ds.cache().prefetch(tf.data.AUTOTUNE)
test_ds

In [ ]:
#papermill_description=sequential_model
loss, accuracy = export_model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: {:2.2%}".format(accuracy))

Use `saved_model.save` to export it.

In [ ]:
#papermill_description=sequential_model_save
tf.saved_model.save(export_model, 'export.tf')

In [ ]:
#papermill_description=sequential_model_export
loaded = tf.saved_model.load('export.tf')

In [ ]:
#papermill_description=sequential_model_export
export_model(tf.constant(['TASK [Kubeflow install (standalone deployment) docker-desktop cluster - wait for crd/applications.app.k8s.io] ***'])).numpy()

In [ ]:
#papermill_description=sequential_model_export
loaded(tf.constant(['TASK [Kubeflow install (standalone deployment) docker-desktop cluster - wait for crd/applications.app.k8s.io] ***'])).numpy()

The loss and accuracy for the model on encoded validation set and the exported model on the raw validation set are the same, as expected.

### Run inference on new data

In [ ]:
inputs = [
    "TASK [Git checkout https://github.com/jannahio/frontend.git] *******************",  # Label: 1
    "TASK [Create images/ubuntu directory] ******************************************",  # Label: 2
    "TASK [Import cleanup prepared build image tasks - streamos middleware and frontend] ***",  # Label: 0
]

predicted_scores = export_model.predict(inputs)
predicted_labels = tf.math.argmax(predicted_scores, axis=1)

for input, label in zip(inputs, predicted_labels):
  print("Log Line: ", input)
  print("Predicted label: ", label.numpy())